---
## IoT-Effizienz der Weihnachtslichter 💡

### 0. Python vorbereiten
Importiere die Libraries `pandas` und `numpy`.

In [1]:
import pandas as pd
import numpy as np

---
### 1. IoT-Simulationsdaten erstellen

Dein IoT-System misst den stündlichen Stromverbrauch (kWh) und die Umgebungshelligkeit (Lux) für eine Woche im Dezember.
Der Verbrauch wird für zwei Lichtertypen simuliert:
* **Alt (Incandescent):** 0.1 kWh pro Stunde
* **Neu (LED):** 0.01 kWh pro Stunde

**Erstelle die DataFrames `consumption` und `sensors` (168 Stunden).**

In [3]:
# Stündliche Daten für 7 Tage (168 Stunden)
n_hours = 7 * 24
timestamps = pd.date_range(start='2025-12-01', periods=n_hours, freq='h')

# 1. Verbrauch (kWh) - Simuliert
consumption = pd.DataFrame({
    'Type': np.random.choice(['Alt (Incandescent)', 'Neu (LED)'], size=n_hours, p=[0.3, 0.7]),
    'On_Status': np.random.randint(0, 2, n_hours) 
}, index=timestamps)

consumption['Factor_kWh'] = consumption['Type'].apply(lambda x: 0.1 if 'Alt' in x else 0.01)

# 2. Sensor (Lux) - Simuliert (dunkel nachts, hell tagsüber)
sensors = pd.DataFrame({
    'Ambient_Lux': np.clip(100 + np.sin(np.linspace(0, 4*np.pi, n_hours)) * 100, 5, 200)
}, index=timestamps)

consumption.index.name = 'Hour'
sensors.index.name = 'Hour'
merged_data = consumption.join(sensors)
merged_data['Total_Consumption'] = merged_data['Factor_kWh'] * merged_data['On_Status']
weekly_consumption = merged_data['Total_Consumption'].sum()

print(f"Originaler wöchentlicher Verbrauch: {weekly_consumption:.2f} kWh")

Originaler wöchentlicher Verbrauch: 4.09 kWh


---
### 2. Effizienz-Optimierung (Funktion)

Die Lichter sollen nur leuchten (`On_Status` = 1), wenn es **wirklich dunkel** ist. Das heißt, nur wenn die Umgebungshelligkeit (`Ambient_Lux`) **unter 50 Lux** liegt.

**Aufgabe:**
1.  **Definiere eine Funktion** (`optimize_schedule`), die den `merged_data`-DataFrame als Input nimmt.
2.  **Erstelle innerhalb der Funktion** eine neue Spalte (`Optimized_Status`), die den `On_Status` auf `1` setzt, **NUR WENN** `Ambient_Lux` < 50 ist. Andernfalls setze sie auf `0`.
3.  **Berechne die Einsparung** (`reduction_kWh`) durch den optimierten Plan.

In [4]:
def optimize_schedule(df):
    # 1. & 2. Optimierten Status erstellen
    df['Optimized_Status'] = np.where(
        (df['On_Status'] == 1) & (df['Ambient_Lux'] < 50),
        1,  # True-Wert
        0   # False-Wert
    )
    
    # Verbrauch mit optimiertem Status berechnen
    df['Optimized_Consumption'] = df['Factor_kWh'] * df['Optimized_Status']
    
    optimized_consumption = df['Optimized_Consumption'].sum()
    return optimized_consumption

optimized_consumption = optimize_schedule(merged_data.copy())
reduction_kWh = weekly_consumption - optimized_consumption

print(f"Einsparung durch IoT-Optimierung: {reduction_kWh:.2f} kWh")

Einsparung durch IoT-Optimierung: 2.70 kWh


---
### 3. Real-World Kontext: EWZ-Datenanalyse

Die Einsparung von {reduction_kWh:.2f} kWh durch einen Haushalt klingt klein, aber was bedeutet das im Kontext der **gesamten Stadt Zürich**? Wir nutzen die bereitgestellten EWZ-Daten zur Stromabgabe (`ewz_stromabgabe_netzebenen_stadt_zuerich.csv`).

**Aufgabe:**
1.  **Lade die EWZ-Daten** und konvertiere die `Timestamp`-Spalte in den Datetime-Typ. Setze sie als Index.
2.  **Finde die maximale 15-Minuten-Verbrauchsspitze** (`Value_NE5`) für eine typische Weihnachtswoche (z.B. vom 2017-12-18 bis 2017-12-24).
3.  **Berechne den prozentualen Anteil** deiner optimierten Einsparung (`reduction_kWh`) an dieser maximalen städtischen Verbrauchsspitze (in kW, nicht kWh!). *Hinweis: Da die EWZ-Werte in kW pro 15-Minuten-Intervall angegeben sind, verwenden wir den Wert direkt als Peak-Power.*

In [9]:
EWZ_FILE = 'ewz_stromabgabe_netzebenen_stadt_zuerich.csv'
ewz_data = pd.read_csv(EWZ_FILE)

# 1. Timestamp konvertieren und als Index setzen
ewz_data['Timestamp'] = pd.to_datetime(ewz_data['Timestamp'])
ewz_data = ewz_data.set_index('Timestamp')

# 2. Filtern für eine Beispiel-Weihnachtswoche (2017)
start_date = '2017-02-18'
end_date = '2017-02-24'
christmas_week_data = ewz_data.loc[start_date:end_date]

# Finde den maximalen Verbrauch (Peak-Power) in kW in Netzebene 5
peak_consumption_city_kW = christmas_week_data['Value_NE5'].max()

# Wir müssen die wöchentliche kWh-Einsparung in eine kW-Einsparung pro Stunde umrechnen, 
# um sie mit dem Peak-kW-Wert der Stadt vergleichen zu können. 
# (Wöchentliche Einsparung / 168 Stunden)
average_saving_kW = reduction_kWh / 168 

# 3. Prozentualen Anteil berechnen
percentage_of_peak = (average_saving_kW / peak_consumption_city_kW) * 100

print(f"Maximaler Verbrauch (Netzebene 5) in Zürich ({start_date} bis {end_date}): {peak_consumption_city_kW:.2f} kW")
print(f"Einsparung als Anteil der städtischen Spitze: {percentage_of_peak:.6f} %")

Maximaler Verbrauch (Netzebene 5) in Zürich (2017-02-18 bis 2017-02-24): 35535.18 kW
Einsparung als Anteil der städtischen Spitze: 0.000045 %
